In [ ]:
# Rock Paper Game

import random
import time
import cv2
import numpy as np
import mediapipe as mp

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7,min_tracking_confidence=0.7)


choices = ["Rock","Paper","Scissors"]
start_time = None
game_started = False
player_move = ""
computer_move = ""
winner = ""


def detect_move(lms):
    finger_tips = [8,12,16,20]
    finger_up = []

    if lms[4][0] > lms[3][0]:
        finger_up.append(1)
    else:
        finger_up.append(0)

    for tip in finger_tips:
        if lms[tip][1] < lms[tip-2][1]:
            finger_up.append(1)
        else:
            finger_up.append(0)
    print(finger_up)
    if finger_up == [0,0,0,0,0]:
        return "Rock"
    elif finger_up == [1,1,1,1,1]:
        return "Paper"
    elif finger_up[1]==1 and finger_up[2]==1 and finger_up[3]==0 and finger_up[4]==0:
        return "Scissors"
    else:
        return "Unknown"


def Decidewinner(player,computer):
    if player==computer:
        return "Tie"
    elif (player=="Rock" and computer=="Scissors") or \
    (player=="Scissors" and computer=="Paper") or \
    (player=="Paper" and computer=="Rock"):
        return "You Win"
    else :
        return "Computer Win"

while cap.isOpened():
    r,f = cap.read()
    if r==False:
        break
    f = cv2.flip(f,1)
    h,w,_ = f.shape


    rgb = cv2.cvtColor(f,cv2.COLOR_BGR2RGB)
    res = hands.process(rgb)

    lms = []
    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            for id,lm in enumerate(hand_landmarks.landmark):
                cx,cy = int(lm.x*w),int(lm.y*h)
                lms.append((cx,cy))
            mp_drawing.draw_landmarks(f,hand_landmarks,mp_hands.HAND_CONNECTIONS)

    key = cv2.waitKey(1)
    if key == ord('s') and not game_started:
        start_time = time.time()
        game_started = True
        player_move = ""
        computer_move = ""
        winner = ""

    if game_started:
        elapsed = time.time() - start_time
        countdown = 3 - int(elapsed)

        if countdown>0:
            cv2.putText(f,str(countdown),(250,250),cv2.FONT_HERSHEY_COMPLEX,1.5,(0,255,0),3)
        else:
            if len(lms)>=21:
                player_move = detect_move(lms)
                computer_move = random.choice(choices)
                winner = Decidewinner(player_move,computer_move)
            game_started = False
    
    if player_move:
        cv2.putText(f,f"You: {player_move}",(10,50),cv2.FONT_HERSHEY_COMPLEX,1.5,(0,255,0),3)
        cv2.putText(f,f"Computer: {computer_move}",(10,100),cv2.FONT_HERSHEY_COMPLEX,1.5,(0,255,0),3)
        cv2.putText(f,f"Winner: {winner}",(10,150),cv2.FONT_HERSHEY_COMPLEX,1.5,(0,255,0),3)

    cv2.putText(f,"Press s to play",(10,450),cv2.FONT_HERSHEY_COMPLEX,1.5,(0,255,0),3)

    cv2.imshow("RPS GAME",f)
    if cv2.waitKey(25) & 0xff == ord('d'):
        break


cap.release()
cv2.destroyAllWindows()